In [ ]:
%%capture
!pip install --upgrade gliner

from huggingface_hub import login
login("")

In [ ]:
from datasets import load_dataset

from gliner import GLiNER
from gliner.data_processing.collator import BiEncoderSpanDataCollator 
from gliner.training import Trainer, TrainingArguments


2025-12-07 15:54:33.816974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765122873.963118      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765122874.011537      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
dataset = load_dataset("nqdhocai/phoner_covid19")

label_names = [
    "SYMPTOM_AND_DISEASE",
    "PATIENT_ID",
    "NAME",
    "AGE",
    "GENDER",
    "JOB",
    "LOCATION",
    "ORGANIZATION",
    "TRANSPORTATION",
    "DATE"
]

def convert_annoctr_format(dataset_split):
    formatted_data = []

    for example in dataset_split:
        tokens = example["words"]
        tags = example["tags"] 

        if not tokens:
            continue

        ner_entities = []
        start_idx = None
        current_label = None

        for idx, tag_str in enumerate(tags):
            if tag_str.startswith("B-"):
                if current_label is not None:
                    ner_entities.append([start_idx, idx - 1, current_label])

                current_label = tag_str[2:] 
                start_idx = idx

            elif tag_str.startswith("I-"):
                new_label = tag_str[2:]
                if current_label is not None and current_label == new_label:
                    continue 
                else:
                    if current_label is not None:
                        ner_entities.append([start_idx, idx - 1, current_label])
                    current_label = None
                    start_idx = None

            else: 
                if current_label is not None:
                    ner_entities.append([start_idx, idx - 1, current_label])
                    current_label = None
                    start_idx = None

        if current_label is not None:
            ner_entities.append([start_idx, len(tokens) - 1, current_label])

        formatted_data.append({
            'tokenized_text': tokens,
            'ner': ner_entities
        })

    return formatted_data

train_data_formatted = convert_annoctr_format(dataset["train"])
test_data_formatted = convert_annoctr_format(dataset["test"])

README.md:   0%|          | 0.00/403 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/72.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9027 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Training samples: 9027
Test samples: 1000

--- Mẫu dữ liệu đầu tiên ---
{'tokenized_text': ['Đồng', 'thời', ',', 'bệnh', 'viện', 'tiếp', 'tục', 'thực', 'hiện', 'các', 'biện', 'pháp', 'phòng', 'chống', 'dịch', 'bệnh', 'COVID', '-', '19', 'theo', 'hướng', 'dẫn', 'của', 'Bộ', 'Y', 'tế', '.'], 'ner': [[23, 25, 'ORGANIZATION']]}


In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
data_collator = BiEncoderSpanDataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

training_args = TrainingArguments(
    output_dir="models/gliner_vietnamese", 
    learning_rate=5e-6,             
    weight_decay=0.01,              
    others_lr=1e-5,                 
    others_weight_decay=0.01,
    lr_scheduler_type="linear",     
    warmup_ratio=0.1,               
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=32,
    num_train_epochs=5,             
    eval_strategy="steps",    
    save_steps=100,                 
    save_total_limit=2,             
    eval_steps=100,                 
    logging_steps=50,               
    report_to="none",               
    use_cpu=False                   
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_formatted,
    eval_dataset=test_data_formatted,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🚀 Bắt đầu huấn luyện...


Step,Training Loss,Validation Loss
100,82.764500,67.590401
200,44.107700,39.579109
300,27.019600,31.534153
400,29.539500,27.218412
500,25.381500,23.271528
600,27.316300,20.593391
700,20.570700,21.096128
800,21.254800,20.497957
900,23.413200,18.705797
1000,20.285000,19.345547


TrainOutput(global_step=1415, training_loss=33.113308618767945, metrics={'train_runtime': 1427.5503, 'train_samples_per_second': 31.617, 'train_steps_per_second': 0.991, 'total_flos': 0.0, 'train_loss': 33.113308618767945, 'epoch': 5.0})